# CLUSTERING NEIGHBOURHOODS IN TORONTO (1 of 3) - LUIS MdP
## Creating the neighbourhood dataframe

In this notebook, I will cluster the neighbourhoods in Toronto as it is stated in the assigment instructions.

First of all, I am going to import the neighbourhood table from Wikipedia, and I am going to proceed with the data wrangling phase.

In [1]:
import pandas as pd
import numpy as np

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
TOR_neigh_list=pd.read_html(url)

In [2]:
TOR_neigh = TOR_neigh_list[0]
TOR_neigh.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


In [3]:
# Retreiving info about the dimension of the dataframe

TOR_neigh.shape

(287, 3)

In [4]:
# Checking that all data is in the correct format
TOR_neigh.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

The next step is to drop from the created dataframe ("TOR_neigh") all the rows with "Not assigned" borough. To do this, first it is necessary to replace "Not assigned" with "NaN" (Python's default missing value marker).

In [5]:
# replace "Not assigned" to NaN
TOR_neigh["Borough"].replace("Not assigned", np.nan, inplace = True)
TOR_neigh.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,NaN,Not assigned
9,M9A,Downtown Toronto,Queen's Park


In [6]:
# drop rows with "NaN" in "Borough"

TOR_neigh.dropna(subset=["Borough"], axis=0, inplace=True)
TOR_neigh.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [7]:
# Retreiving info about the dimension of the new dataframe

TOR_neigh.shape

(210, 3)

In [8]:
# Grouping Neighbourhood in every unique Postcode

TOR_neigh = TOR_neigh.groupby(["Postcode","Borough"])["Neighbourhood"].apply(list)
TOR_neigh = TOR_neigh.sample(frac=1).reset_index()
TOR_neigh["Neighbourhood"]= TOR_neigh["Neighbourhood"].str.join(", ")
TOR_neigh.head(10)

,Postcode,Borough,Neighbourhood
0,M1R,Scarborough,"Maryvale, Wexford"
1,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
2,M3J,North York,"Northwood Park, York University"
3,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
4,M5E,Downtown Toronto,Berczy Park
5,M6C,York,Humewood-Cedarvale
6,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
7,M1G,Scarborough,Woburn
8,M1E,Scarborough,"Guildwood, Morningside, West Hill"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
# Retreiving info about the dimension of the new dataframe

TOR_neigh.shape

(103, 3)

In [10]:
# replace "Not assigned" in Neighbourhood with the name of Borough

TOR_neigh["Neighbourhood"].replace("Not assigned", TOR_neigh["Borough"], inplace = True)
TOR_neigh.head(10)

,Postcode,Borough,Neighbourhood
0,M1R,Scarborough,"Maryvale, Wexford"
1,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
2,M3J,North York,"Northwood Park, York University"
3,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
4,M5E,Downtown Toronto,Berczy Park
5,M6C,York,Humewood-Cedarvale
6,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
7,M1G,Scarborough,Woburn
8,M1E,Scarborough,"Guildwood, Morningside, West Hill"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [11]:
# Confirmation the replace has been done

TOR_neigh[TOR_neigh.Postcode=="M7A"]

,Postcode,Borough,Neighbourhood
86,M7A,Queen's Park,Queen's Park


In [12]:
# Confirmation there is no Neighbourhood with "Not assigned" value

TOR_neigh[TOR_neigh["Neighbourhood"]=="Not assigned"]


,Postcode,Borough,Neighbourhood


In [13]:
# Retreiving info about the dimension of the new dataframe

TOR_neigh.shape

(103, 3)

# CLUSTERING NEIGHBOURHOODS IN TORONTO (2 of 3) - LUIS MdP
## Retreiving the coordinates of each neighbourhood

In this part, I am going to create a new dataframe with the latitude and longitude of each neighbourhood, using geocoder.

In [14]:
# install geocoder:
!conda install -c conda-forge geocoder --yes
print('geocoder installed!')

# install folium:
!conda install -c conda-forge folium=0.5.0
print('folium installed!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

In [15]:
import requests
import geocoder # to retrieve latitude and longitude
import csv
import json
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [16]:
# Trying to use geocoder... without success

#lat_long = []

# Getting the Postal code

# for i in range(TOR_neigh.shape[0]) :
#     postal_code = TOR_neigh.loc[i, "Postcode"]
# 
# initialize the coordinates to None

# lat_long_coor = None

# loop until you get the coordinates

# while(lat_long_coor is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#     lat_long_coor = g.latlng
#     lat_long.append(lat_long_coor)

As geocoder is not working properly, I will use the geospatial info in the cvs file provided.

In [17]:
csv = 'http://cocl.us/Geospatial_data'
TOR_neigh_geo=pd.read_csv(csv)
TOR_neigh_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
TOR_neigh_geo.shape

(103, 3)

In [20]:
TOR_neigh_geo.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In order to merge properly TOR_neigh_geo and TOR_neigh, I rename the column "Postal Code" in TOR_neigh_geo with the name in TOR_neigh ("Postcode").

In [21]:
TOR_neigh_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [22]:
TOR_neigh_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
TOR_neigh_final = TOR_neigh.merge(TOR_neigh_geo, on="Postcode")
TOR_neigh_final.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
1,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
2,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
3,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
6,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
7,M1G,Scarborough,Woburn,43.770992,-79.216917
8,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# CLUSTERING NEIGHBOURHOODS IN TORONTO (3 of 3) - LUIS MdP
## Exploring and clustering neighbourhoods

First of all, I import the rest of the libraries needed.

In [25]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


I am using only the Boroughs that contains 'Toronto'.

In [27]:
TOR_bor = TOR_neigh_final[TOR_neigh_final['Borough'].str.contains('Toronto')]
TOR_bor.reset_index(drop=True, inplace=True)
TOR_bor.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M4M,East Toronto,Studio District,43.659526,-79.340923
5,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
6,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
7,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
8,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [28]:
TOR_bor.shape

(39, 5)

In [29]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TOR_bor['Borough'].unique()),
        TOR_bor.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [30]:
#Following the same as I did with NY

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [33]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(TOR_bor['Latitude'], TOR_bor['Longitude'], TOR_bor['Borough'], TOR_bor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now I will use FourSquare query to cluster neighborhoods

In [35]:
CLIENT_ID = 'K23ALSECZRLBI0OTEWJ1RGI5M05W2NS0P0RBSCICVWWYCEWV'
CLIENT_SECRET = 'OVBZVBAHG1ILIZJ0RCPRPIV4ZRK2WA4MNWPLRT5SQUOY53PR'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K23ALSECZRLBI0OTEWJ1RGI5M05W2NS0P0RBSCICVWWYCEWV
CLIENT_SECRET:OVBZVBAHG1ILIZJ0RCPRPIV4ZRK2WA4MNWPLRT5SQUOY53PR


Now, let's get the top 100 venues that are in every borough in Toronto within a radius of 500 meters.

In [81]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
TOR_venues = getNearbyVenues(names=TOR_bor['Neighbourhood'],
                                   latitudes=TOR_bor['Latitude'],
                                   longitudes=TOR_bor['Longitude']
                                  )

The Annex, North Midtown, Yorkville
Berczy Park
Business Reply Mail Processing Centre 969 Eastern
Harbourfront
Studio District
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Queen's Park
Church and Wellesley
Commerce Court, Victoria Hotel
Davisville
The Danforth West, Riverdale
Brockton, Exhibition Place, Parkdale Village
North Toronto West
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Runnymede, Swansea
Dovercourt Village, Dufferin
Little Portugal, Trinity
Christie
The Beaches
Davisville North
Lawrence Park
Parkdale, Roncesvalles
Central Bay Street
Ryerson, Garden District
Harbourfront East, Toronto Islands, Union Station
Cabbagetown, St. James Town
Rosedale
Roselawn
Design Exchange, Toronto Dominion Centre
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Forest Hill North, Forest Hill West
Harbord, University of Toronto
The Beaches West, India Bazaar
Moore Park, Summerhill East
St. J

In [83]:
TOR_venues.shape

(1685, 7)

In [84]:
TOR_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
1,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
2,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
3,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant
4,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Big Crow,43.675896,-79.403680,BBQ Joint
5,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Creeds Coffee Bar,43.674100,-79.410838,Coffee Shop
6,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Madame Boeuf And Flea,43.675240,-79.406620,Burger Joint
7,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Dish Cooking Studio,43.674066,-79.410764,Café
8,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Jean Sibelius Square,43.671426,-79.408831,Park
9,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Pour House,43.675641,-79.403821,Pub


In [85]:
TOR_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17


In [86]:
print('There are {} uniques categories.'.format(len(TOR_venues['Venue Category'].unique())))

There are 232 uniques categories.


Analyzing each neighbourhood

In [152]:
# one hot encoding
TOR_onehot = pd.get_dummies(TOR_venues[['Venue Category']], prefix="", prefix_sep="")
TOR_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [153]:
# rename column 'Neighborhood' -> 'Neighborhood_onehot':
TOR_onehot.rename(columns = {'Neighborhood':'Neighborhood_onehot'}, inplace = True)

# add neighborhood column back to dataframe
TOR_onehot.insert(loc=0, column='Neighborhood', value=TOR_venues['Neighborhood']) 
TOR_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood_onehot,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [154]:
TOR_onehot.drop(['Neighborhood_onehot'], axis='columns', inplace=True)
TOR_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"The Annex, North Midtown, Yorkville",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [155]:
TOR_onehot.shape

(1685, 232)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [156]:
TOR_grouped = TOR_onehot.groupby('Neighborhood').mean().reset_index()
TOR_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.010000,0.000000,0.030000,0.000000,0.0000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.00,0.000000,0.050000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000

In [157]:
TOR_grouped.shape

(38, 232)

Let's print each neighborhood along with the top 5 most common venues

In [158]:
num_top_venues = 5

for hood in TOR_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = TOR_grouped[TOR_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4              Bar  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1            Beer Bar  0.04
2  Seafood Restaurant  0.04
3          Steakhouse  0.04
4      Farmers Market  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3   Grocery Store  0.05
4          Bakery  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0           Brewery  0.06
1     Burrito Place  0.06
2  Recording Studio  0.06
3            Garden  0.06
4     Garden Center  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0       Airport Lounge  0.13
1      Airport Service  0.13
2     Ai

Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [159]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [160]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
TOR_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
TOR_neighborhoods_venues_sorted['Neighborhood'] = TOR_grouped['Neighborhood']

for ind in np.arange(TOR_grouped.shape[0]):
    TOR_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TOR_grouped.iloc[ind, :], num_top_venues)
    
TOR_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Salad Place,Burger Joint,Bar,Bakery,Sushi Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Beer Bar,Cocktail Bar,Steakhouse,Café,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant,Bistro
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Gym,Restaurant,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
3,Business Reply Mail Processing Centre 969 Eastern,Burrito Place,Auto Workshop,Restaurant,Skate Park,Comic Shop,Garden Center,Gym / Fitness Center,Garden,Brewery,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Sculpture Garden,Rental Car Location,Bar,Harbor / Marina,Airport Gate


In [178]:
TOR_neighborhoods_venues_sorted.shape

(38, 12)

## Clustering Neighborhoods

Looking at Toronto map, it seems that running *k*-means to cluster the neighborhood into 5 clusters is enough. Considering 6 or 7 clusters does not seem to add more info and it would add complexity to the study.

In [179]:
# set number of clusters
kclusters = 5

TOR_grouped_clustering = TOR_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TOR_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       2, 0, 4, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [181]:
TOR_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
TOR_neighborhoods_venues_sorted.head(10)

ValueError: cannot insert Cluster Labels, already exists

In [182]:
TOR_neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Salad Place,Burger Joint,Bar,Bakery,Sushi Restaurant,Asian Restaurant
1,0,Berczy Park,Coffee Shop,Beer Bar,Cocktail Bar,Steakhouse,Café,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant,Bistro
2,0,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Gym,Restaurant,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
3,0,Business Reply Mail Processing Centre 969 Eastern,Burrito Place,Auto Workshop,Restaurant,Skate Park,Comic Shop,Garden Center,Gym / Fitness Center,Garden,Brewery,Smoke Shop
4,0,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Sculpture Garden,Rental Car Location,Bar,Harbor / Marina,Airport Gate
5,0,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Flower Shop,Italian Restaurant,Café,Bakery,Pub,Pizza Place,Grocery Store,Pet Store
6,0,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Burger Joint,Ice Cream Shop,Café,Japanese Restaurant,Fried Chicken Joint,Chinese Restaurant,Salad Place
7,0,"Chinatown, Grange Park, Kensington Market",Café,Bar,Coffee Shop,Vietnamese Restaurant,Bakery,Mexican Restaurant,Chinese Restaurant,Dumpling Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar
8,0,Christie,Grocery Store,Café,Park,Restaurant,Baby Store,Athletics & Sports,Candy Store,Diner,Italian Restaurant,Nightclub
9,0,Church and Wellesley,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Gastropub,Café,Hotel,Mediterranean Restaurant,Gym


In [183]:
TOR_merged = TOR_bor
TOR_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M4M,East Toronto,Studio District,43.659526,-79.340923
5,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
6,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
7,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
8,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [184]:
TOR_merged.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
TOR_merged.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M4M,East Toronto,Studio District,43.659526,-79.340923


In [185]:
TOR_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M4M,East Toronto,Studio District,43.659526,-79.340923
5,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
6,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
7,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
8,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [186]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhoo

TOR_merged = TOR_merged.join(TOR_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [187]:
TOR_merged.shape

(39, 16)

In [188]:
TOR_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0.0,Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,BBQ Joint,Indian Restaurant,History Museum,Burger Joint,Park
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.0,Coffee Shop,Beer Bar,Cocktail Bar,Steakhouse,Café,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant,Bistro
2,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0.0,Burrito Place,Auto Workshop,Restaurant,Skate Park,Comic Shop,Garden Center,Gym / Fitness Center,Garden,Brewery,Smoke Shop
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Dessert Shop
4,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place,Brewery,Cheese Shop
5,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0.0,Coffee Shop,Restaurant,Café,Beer Bar,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Hotel,Park,Farmers Market
6,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.0,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Bakery,Deli / Bodega,American Restaurant,Asian Restaurant,Bar
7,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Gastropub,Café,Hotel,Mediterranean Restaurant,Gym
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0.0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Steakhouse,Bakery,Deli / Bodega,Gastropub


In [191]:
# Drop row if NaN in Cluster Label

TOR_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)

TypeError: 'DataFrame' object is not callable

In [192]:
TOR_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0.0,Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,BBQ Joint,Indian Restaurant,History Museum,Burger Joint,Park
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.0,Coffee Shop,Beer Bar,Cocktail Bar,Steakhouse,Café,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant,Bistro
2,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0.0,Burrito Place,Auto Workshop,Restaurant,Skate Park,Comic Shop,Garden Center,Gym / Fitness Center,Garden,Brewery,Smoke Shop
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Dessert Shop
4,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place,Brewery,Cheese Shop
5,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0.0,Coffee Shop,Restaurant,Café,Beer Bar,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Hotel,Park,Farmers Market
6,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.0,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Bakery,Deli / Bodega,American Restaurant,Asian Restaurant,Bar
8,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Gastropub,Café,Hotel,Mediterranean Restaurant,Gym
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0.0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Steakhouse,Bakery,Deli / Bodega,Gastropub
10,M4S,Central Toronto,Davisville,43.704324,-79.388790,0.0,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Costume Shop,Brewery


In [193]:
TOR_merged.columns

Index(['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
       'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

Finally, let's visualize the resulting clusters

In [195]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TOR_merged['Latitude'], TOR_merged['Longitude'], TOR_merged['Neighborhood'], TOR_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters